In [ ]:
## if you wanna make interactive plots in Jupyter notebooks
#pip install ipympl
### Install if needed: pip install ipympl
## after importing matplotlib, run:
#%matplotlib widget


In [ ]:
### importing libraries and suppressing warnings
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import geopandas as gpd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import HDBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
import math

warnings.filterwarnings("ignore")

Exploring the dataset 

In [ ]:
### Defining file path and reading data
path = Path('C:/Users/mailp/Documents/tu_dresden_lectures/DWD_Phaenologie_Apfel_fruehe_Reife_Bluetebeginn_Climate_2023-2025.csv')
### Reading the CSV file into a DataFrame
df = pd.read_csv(path)
### Displaying information about the DataFrame
df.info()

In [ ]:
### lets check the fist 5 rows
df.head()

In [ ]:
### selecting only data for the year 2023 and relevant features for clustering
cluster_df = df[df.Year==2023]
features = ['rad_global_apr', 'soil_moist_apr', 'precip_apr', 'temp_max_apr', 'temp_min_apr', 'Apfelbluete','Hoehe']

X = cluster_df[features].dropna()
 

In [ ]:
### ploting features
### Link for shapefiles
"""https://www.naturalearthdata.com/downloads/110m-cultural-vectors/"""
### Load Germany shapefile
shp_path = Path('C:/Users/mailp/Documents/tu_dresden_lectures/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')
germany = gpd.read_file(shp_path)
germany = germany[germany['NAME'] == 'Germany']

# Create GeoDataFrame from df
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['Lon'], df['Lat']),
    crs="EPSG:4326"
)

# Plot each feature as a separate map over Germany with normalized colorbars

fig, axes = plt.subplots(3, 3, figsize=(18, 10))
axes = axes.flatten()

for i, feature in enumerate(features):
    ax = axes[i]
    germany.plot(ax=ax, color='white', edgecolor='black')
    vmin = gdf[feature].min()
    vmax = gdf[feature].max()
    gdf.plot(
        ax=ax,
        column=feature,
        cmap='jet',
        markersize=8,
        legend=True,
        alpha=0.8,
        vmin=vmin,
        vmax=vmax
    )
    ax.set_title(feature)
    ax.set_axis_off()

plt.tight_layout()
plt.show()

Pre-processing data for clustering

In [ ]:
### scaling data
def scale_data(X):
    scaler = StandardScaler()
    return  scaler.fit_transform(X) 

### data dimension reduction using PCA
def reduce_dimensions(X_scaled, n_components=2, var_threshold=0.85,plot=False):
    # First fit PCA with all components
    pca = PCA(svd_solver='full')
    pca.fit(X_scaled)
    
    # Calculate cumulative variance ratio
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    
    # Find number of components needed for threshold
    n_components_auto = np.argmax(cumsum >= var_threshold) + 1
    
    # Print variance information
    print(f"Total variance explained by {n_components} components: {cumsum[n_components-1]:.3f}")
    print(f"Components needed for {var_threshold:.1%} variance: {n_components_auto}")
    
    # Create final PCA with specified components
    pca = PCA(n_components=n_components, svd_solver='full')
    X_pca = pca.fit_transform(X_scaled)
    
    if plot:
        # Plot variance explained
        plt.figure(figsize=(10, 4))
        plt.plot(range(1, len(cumsum) + 1), cumsum, 'bo-')
        plt.axhline(y=var_threshold, color='r', linestyle='--', label=f'{var_threshold:.1%} threshold')
        plt.axvline(x=n_components, color='g', linestyle='--', label=f'Current n_components={n_components}')
        plt.xlabel('Number of Components')
        plt.ylabel('Cumulative Variance Explained')
        plt.title('PCA Variance Explained')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    return pca, X_pca


In [ ]:
### Scaling the features
X_scaled = scale_data(X) 

In [ ]:
### Reducing dimensions and plotting variance explained for different n_components
for i in range(2,7):
    pca, X_pca = reduce_dimensions(X_scaled, n_components=i, var_threshold=0.85,plot=True)

In [ ]:
### Lets use n_components=3 for further analysis
pca, X_pca = reduce_dimensions(X_scaled, n_components=3,var_threshold=0.85)

In [ ]:
# alternative distance functions
#from scipy.spatial.distance import euclidean, cosine, cityblock, chebyshev

def euclidean_dist(point1, point2):
    """Calculate Euclidean distance using numpy."""
    return np.sqrt(np.sum((point1 - point2) ** 2))

def manhattan_dist(point1, point2):
    """Calculate Manhattan (cityblock) distance using numpy."""
    return np.sum(np.abs(point1 - point2))

def cosine_dist(point1, point2):
    """Calculate Cosine distance using numpy."""
    dot_product = np.dot(point1, point2)
    norms = np.linalg.norm(point1) * np.linalg.norm(point2)
    return 1 - dot_product / norms

def chebyshev_dist(point1, point2):
    """Calculate Chebyshev distance using numpy."""
    return np.max(np.abs(point1 - point2))

def chord_dist(point1, point2):
    """Calculate Chord distance using numpy."""
    dot_product = np.dot(point1, point2)
    norms = np.linalg.norm(point1) * np.linalg.norm(point2)
    return np.sqrt(2 * (1 - dot_product / norms))

# Update the plot_distances function to use these functions
def plot_distances(point1, point2):
    # Calculate distances using the new functions
    metrics = [
        ('Euclidean', euclidean_dist(point1, point2)),
        ('Manhattan', manhattan_dist(point1, point2)),
        ('Cosine', cosine_dist(point1, point2)),
        ('Chebyshev', chebyshev_dist(point1, point2)),
    ]
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    # Plot points in each subplot
    for i, (metric_name, value) in enumerate(metrics):
        ax = axes[i]
        
        # Plot points
        ax.scatter([point1[0], point2[0]], [point1[1], point2[1]], c='blue')
        
        # Annotate points
        ax.annotate('Point 1', (point1[0], point1[1]), xytext=(10, 10), 
                   textcoords='offset points')
        ax.annotate('Point 2', (point2[0], point2[1]), xytext=(10, 10), 
                   textcoords='offset points')
        
        # Visualize distance calculation
        if metric_name == 'Euclidean':
            # Draw direct line
            ax.plot([point1[0], point2[0]], [point1[1], point2[1]], 'r--')
            
        elif metric_name == 'Manhattan':
            # Draw Manhattan path
            ax.plot([point1[0], point2[0]], [point1[1], point1[1]], 'r--')
            ax.plot([point2[0], point2[0]], [point1[1], point2[1]], 'r--')
            
        elif metric_name == 'Chebyshev':
            # Draw box showing maximum distance
            width = abs(point2[0] - point1[0])
            height = abs(point2[1] - point1[1])
            # ax.add_patch(plt.Rectangle((min(point1[0], point2[0]), 
            #                           min(point1[1], point2[1])), 
            #                          width, height, fill=False, 
            #                          linestyle='--', color='red'))
            if width > height:
                # project point2 onto horizontal line through point1 and draw horizontal connector
                proj = np.array([point2[0], point1[1]])
                # dominant horizontal segment (illustrates Chebyshev = width)
                ax.plot([point1[0], proj[0]], [point1[1], proj[1]], 'r--')
                # faint vertical connector from projection to the actual second point
                ax.plot([proj[0], point2[0]], [proj[1], point2[1]], 'r--', alpha=0.5)
            else:
                # project point2 onto vertical line through point1 and draw vertical connector
                proj = np.array([point1[0], point2[1]])
                # dominant vertical segment (illustrates Chebyshev = height)
                ax.plot([point1[0], proj[0]], [point1[1], proj[1]], 'r--')
                # faint horizontal connector from projection to the actual second point
                ax.plot([proj[0], point2[0]], [proj[1], point2[1]], 'r--', alpha=0.5)

            # highlight the projection
            ax.scatter([proj[0]], [proj[1]], c='red', zorder=5)
            ax.annotate('Projection', (proj[0], proj[1]), xytext=(5, -10),
                        textcoords='offset points', color='red')
            
        elif metric_name in ['Cosine']:
            # Draw vectors from origin
            ax.plot([0, point1[0]], [0, point1[1]], 'r--')
            ax.plot([0, point2[0]], [0, point2[1]], 'r--')
            # Draw arc between vectors for Chord distance
           
        ax.set_title(f'{metric_name}\nDistance: {value:.3f}')
        ax.grid(True)
        ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
        ax.axvline(x=0, color='k', linestyle='-', alpha=0.3)
        ax.set_aspect('equal')
    
    # Remove extra subplot
    axes[-1].remove()
    axes[-2].remove()
    
    plt.tight_layout()
    plt.show()

# Take two random points from PCA data (first two components)
np.random.seed(42)  # for reproducibility
idx1, idx2 = np.random.randint(0, len(X_pca), 2)
point1 = X_pca[idx1, :2]
point2 = X_pca[idx2, :2]

# Plot distances
plot_distances(point1, point2)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 
def plot_pca_scatter(X_pca, pca, features, color_labels=None,
                     figsize=(8,6), elev=30, azim=120,
                     s=35, alpha=0.8, cmap='viridis'):
   
    n_comp = X_pca.shape[1]
    
    # scale factor for arrows so they’re visible but not huge
    arrow_scale = np.max(np.abs(X_pca)) * 1.5

    if n_comp == 2:
        fig, ax = plt.subplots(figsize=figsize)

        # Scatter points
        sc = ax.scatter(X_pca[:, 0], X_pca[:, 1],
                        c=color_labels, cmap=cmap, s=s, alpha=alpha, edgecolor='k')

        # Feature loadings (arrows)
        for i, feature in enumerate(features):
            ax.arrow(0, 0,
                     pca.components_[0, i]*arrow_scale,
                     pca.components_[1, i]*arrow_scale,
                     color='r', alpha=0.6, head_width=0.05)
            ax.text(pca.components_[0, i]*arrow_scale*1.15,
                    pca.components_[1, i]*arrow_scale*1.15,
                    feature, color='r', fontsize=10, ha='center')

        ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
        ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
        ax.set_title("PCA Biplot (2D)")

        if color_labels is not None:
            plt.colorbar(sc, ax=ax, label='Cluster/Label')

        ax.grid(True)
        plt.tight_layout()
        plt.show()

    elif n_comp == 3:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(projection='3d')

        sc = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
                        c=color_labels, cmap=cmap, s=s, alpha=alpha)

        # 3D arrows (feature loadings)
        for i, feature in enumerate(features):
            ax.quiver(0, 0, 0,
                      pca.components_[0, i]*arrow_scale,
                      pca.components_[1, i]*arrow_scale,
                      pca.components_[2, i]*arrow_scale,
                      color='r', alpha=0.6)
            ax.text(pca.components_[0, i]*arrow_scale,
                    pca.components_[1, i]*arrow_scale,
                    pca.components_[2, i]*arrow_scale,
                feature, color='r', fontsize=10, ha='center',va='center')

        ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
        ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
        ax.set_zlabel(f"PC3 ({pca.explained_variance_ratio_[2]*100:.1f}% var)")
        ax.set_title("PCA Biplot (3D)")
        ax.view_init(elev=elev, azim=azim)
#
        if color_labels is not None:
            fig.colorbar(sc, ax=ax, label='Cluster/Label', pad=0.1)

        plt.show()

    else:
        raise ValueError(f"PCA must have 2 or 3 components, got {n_comp}")


In [ ]:
### Lets plot PCA scatter
plot_pca_scatter(X_pca,pca, features)

Starting with K-means

In [ ]:
def find_best_k_kmeans(X, k_min=2, k_max=10, random_state=0):
    results = []
    for k in range(k_min, k_max+1):
        km = KMeans(n_clusters=k, random_state=random_state, n_init=10)
        labels = km.fit_predict(X)
        # silhouette requires at least 2 clusters and less than n_samples clusters
        try:
            sil = silhouette_score(X, labels)
        except Exception:
            sil = np.nan
        try:
            db = davies_bouldin_score(X, labels)
        except Exception:
            db = np.nan
        results.append({"k": k, "inertia": km.inertia_, "silhouette": sil, "davies_bouldin": db})
    res_df = pd.DataFrame(results)
    # choose best k by highest silhouette_score (fall back to lowest davies_bouldin)
    if res_df["silhouette"].notna().any():
        best_row = res_df.loc[res_df["silhouette"].idxmax()]
        selection_reason = "silhouette"
    else:
        best_row = res_df.loc[res_df["davies_bouldin"].idxmin()]
        selection_reason = "davies_bouldin"
    return res_df, int(best_row["k"]), selection_reason

from sklearn.metrics import adjusted_rand_score
from sklearn.metrics.cluster import contingency_matrix

def bootstrap_cluster_stability(X, orig_labels, n_clusters, n_bootstrap=100, random_state=0):
    rng = np.random.RandomState(random_state)
    n = X.shape[0]
    ari_scores = []
    unique_orig = np.unique(orig_labels)
    jaccard_per_cluster = {c: [] for c in unique_orig}

    for b in range(n_bootstrap):
        # bootstrap sample indices (with replacement)
        boot_idx = rng.choice(n, size=n, replace=True)
        # fit KMeans on bootstrap sample
        km = KMeans(n_clusters=n_clusters, n_init=10, random_state=random_state + b)
        km.fit(X[boot_idx])
        # predict labels for the full dataset using bootstrap-fitted centers
        pred_labels = km.predict(X)

        # overall stability: Adjusted Rand Index
        ari = adjusted_rand_score(orig_labels, pred_labels)
        ari_scores.append(ari)

        # cluster-wise stability: Jaccard after matching predicted labels to original clusters
        cm = contingency_matrix(orig_labels, pred_labels)  # rows: orig clusters, cols: predicted clusters
        for i, orig_c in enumerate(unique_orig):
            row = cm[i, :]
            if row.sum() == 0:
                jaccard_per_cluster[orig_c].append(0.0)
                continue
            # match predicted label with maximum overlap for this original cluster
            j = row.argmax()
            inter = row[j]
            size_orig = row.sum()
            size_pred = cm[:, j].sum()
            union = size_orig + size_pred - inter
            jacc = inter / union if union > 0 else 0.0
            jaccard_per_cluster[orig_c].append(jacc)

    # summarize results
    summary = {
        "n_bootstrap": n_bootstrap,
        "ari_mean": float(np.mean(ari_scores)),
        "ari_std": float(np.std(ari_scores)),
        "ari_scores": ari_scores,       
        "per_cluster_jaccard_mean": {int(k): float(np.mean(v)) for k, v in jaccard_per_cluster.items()},
        "per_cluster_jaccard_std": {int(k): float(np.std(v)) for k, v in jaccard_per_cluster.items()}
    }
    return summary


In [ ]:
results_df_kmeans, best_k_kmeans, reason_kmeans = find_best_k_kmeans(X_pca, 2, 10)
print(f"Best k by {reason_kmeans}: {best_k_kmeans}")
results_df_kmeans

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
# # Assign clusters back to the DataFrame (NaNs will be dropped!)
df_clustered = cluster_df.copy()
kmeans_labels = kmeans.fit_predict(X_pca)
df_clustered['cluster'] = kmeans.fit_predict(X_pca)

In [ ]:
# Run bootstrap stability check on KMeans result (uses X_pca and km_labels already computed)
stability = bootstrap_cluster_stability(X_pca, kmeans_labels, n_clusters=2, n_bootstrap=50, random_state=9)

print("Bootstrap stability (KMeans):")
print(f"  Bootstraps: {stability['n_bootstrap']}")
print(f"  ARI mean ± std: {stability['ari_mean']:.4f} ± {stability['ari_std']:.4f}")
print("  Per-cluster Jaccard mean ± std:")
for c in sorted(stability['per_cluster_jaccard_mean'].keys()):
    jm = stability['per_cluster_jaccard_mean'][c]
    js = stability['per_cluster_jaccard_std'][c]
    print(f"    Cluster {c}: {jm:.3f} ± {js:.3f}")

Now, HDBSCAN

In [ ]:
param_grid = {
    'min_cluster_size': [5, 10, 15, 20, 30, 50, 70,100],
    'min_samples': [5, 10, 15, 20, 30,50],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan', 'cosine']
}

# Store results
results = []

# Grid search
for min_cluster_size in param_grid['min_cluster_size']:
    for min_samples in param_grid['min_samples']:
        for algorithm in param_grid['algorithm']:
            for metric in param_grid['metric']:
                try:
                    # Initialize and fit HDBSCAN
                    clusterer = HDBSCAN(
                        min_cluster_size=min_cluster_size,
                        min_samples=min_samples,
                        algorithm=algorithm,
                        metric=metric
                    )
                    labels = clusterer.fit_predict(X_pca)
                    
                    # Only evaluate if we have at least 2 clusters (excluding noise)
                    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                    if n_clusters >= 2:
                        # Calculate scores
                        sil_score = silhouette_score(X_pca, labels)
                        noise_ratio = (labels == -1).sum() / len(labels)
                        
                        results.append({
                            'min_cluster_size': min_cluster_size,
                            'min_samples': min_samples,
                            'algorithm': algorithm,
                            'metric': metric,
                            'n_clusters': n_clusters,
                            'silhouette': sil_score,
                            'noise_ratio': noise_ratio
                        })
                except:
                    continue

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Find best parameters
best_result = results_df.sort_values('silhouette', ascending=False).iloc[0]

print("\nBest Parameters:")
print(f"min_cluster_size: {best_result['min_cluster_size']}")
print(f"min_samples: {best_result['min_samples']}")
print(f"algorithm: {best_result['algorithm']}")
print(f"metric: {best_result['metric']}")
print(f"\nResults:")
print(f"Number of clusters: {best_result['n_clusters']}")
print(f"Silhouette score: {best_result['silhouette']:.3f}")
print(f"Noise ratio: {best_result['noise_ratio']:.2%}")

# Plot top 10 configurations
plt.figure(figsize=(10, 6))
top_10 = results_df.nlargest(10, 'silhouette')
plt.bar(range(10), top_10['silhouette'])
plt.xlabel('Configuration rank')
plt.ylabel('Silhouette score')
plt.title('Top 10 HDBSCAN configurations')
plt.tight_layout()
plt.show()

In [ ]:
hdb = HDBSCAN(min_cluster_size=50,
                    min_samples=5,
                    metric= 'cosine',
                    algorithm='brute')
hdb.fit(X_pca)

df_clustered_h = cluster_df.copy()

df_clustered_h['cluster'] = hdb.labels_

final_hdbscan = df_clustered_h[df_clustered_h['cluster'] != -1]  # Remove noise points if any

In [ ]:
def bootstrap_hdbscan_stability(X, orig_labels, n_bootstrap=100, random_state=0, **hdbscan_params):
    rng = np.random.RandomState(random_state)
    n = X.shape[0]
    ari_scores = []
    unique_orig = np.unique(orig_labels[orig_labels != -1])  # Exclude noise points
    jaccard_per_cluster = {c: [] for c in unique_orig}

    for b in range(n_bootstrap):
        # bootstrap sample indices (with replacement)
        boot_idx = rng.choice(n, size=n, replace=True)
        
        # fit HDBSCAN on bootstrap sample
        clusterer = HDBSCAN(**hdbscan_params)
        boot_labels = clusterer.fit_predict(X[boot_idx])
        
        # predict labels for the full dataset
        pred_labels = clusterer.fit_predict(X)

        # Calculate ARI only for non-noise points
        mask_orig = orig_labels != -1
        mask_pred = pred_labels != -1
        if np.any(mask_orig) and np.any(mask_pred):
            ari = adjusted_rand_score(orig_labels[mask_orig], pred_labels[mask_orig])
            ari_scores.append(ari)

        # cluster-wise stability using Jaccard
        cm = contingency_matrix(orig_labels, pred_labels)
        for i, orig_c in enumerate(unique_orig):
            row = cm[int(orig_c), :]
            if row.sum() == 0:
                jaccard_per_cluster[orig_c].append(0.0)
                continue
            j = row.argmax()
            inter = row[j]
            size_orig = row.sum()
            size_pred = cm[:, j].sum()
            union = size_orig + size_pred - inter
            jacc = inter / union if union > 0 else 0.0
            jaccard_per_cluster[orig_c].append(jacc)

    # summarize results
    summary = {
        "n_bootstrap": n_bootstrap,
        "ari_mean": float(np.mean(ari_scores)) if ari_scores else 0.0,
        "ari_std": float(np.std(ari_scores)) if ari_scores else 0.0,
        "per_cluster_jaccard_mean": {int(k): float(np.mean(v)) for k, v in jaccard_per_cluster.items()},
        "per_cluster_jaccard_std": {int(k): float(np.std(v)) for k, v in jaccard_per_cluster.items()},
        "noise_ratio": float(np.mean(orig_labels == -1))
    }
    return summary

# Run stability analysis with the best HDBSCAN parameters
stability = bootstrap_hdbscan_stability(
    X_pca, 
    hdb.labels_, 
    n_bootstrap=50, 
    random_state=485,
    **hdb.get_params()
)

print("Bootstrap stability (HDBSCAN):")
print(f"  Bootstraps: {stability['n_bootstrap']}")
print(f"  ARI mean ± std: {stability['ari_mean']:.4f} ± {stability['ari_std']:.4f}")
print(f"  Noise ratio: {stability['noise_ratio']:.2%}")
print("  Per-cluster Jaccard mean ± std:")
for c in sorted(stability['per_cluster_jaccard_mean'].keys()):
    jm = stability['per_cluster_jaccard_mean'][c]
    js = stability['per_cluster_jaccard_std'][c]
    print(f"    Cluster {c}: {jm:.3f} ± {js:.3f}")

Lets see now the results

In [ ]:
plot_pca_scatter(X_pca,pca, features, color_labels=kmeans_labels,cmap='jet')
plot_pca_scatter(X_pca,pca, features, color_labels=hdb.labels_,cmap='jet')

In [ ]:
plot_pca_scatter(X_pca[:,0:2],pca, features, color_labels=kmeans_labels,cmap='jet')
plot_pca_scatter(X_pca[:,0:2],pca, features, color_labels=hdb.labels_,cmap='jet')

In [ ]:
from scipy.spatial import ConvexHull
from itertools import cycle
from matplotlib import cm
from sklearn.neighbors import KNeighborsClassifier

# Create DataFrame with PCA components and cluster labels
df_cluster_all = pd.DataFrame({
    'pca1': X_pca[:, 0],
    'pca2': X_pca[:, 1], 
    'pca3': X_pca[:, 2],
    'kmeans_label': kmeans_labels,
    'hdbscan_label': hdb.labels_
})

# Remove rows where hdbscan_label is -1
df_cluster_all = df_cluster_all[df_cluster_all.hdbscan_label != -1]


fig, axes = plt.subplots(1, 2, figsize=(14, 6))
cluster_methods = ['kmeans_label', 'hdbscan_label']
titles = ['KMeans Clusters', 'HDBSCAN Clusters']

for ax, label_col, title in zip(axes, cluster_methods, titles):
    X = df_cluster_all[['pca1', 'pca2']].values
    y = df_cluster_all[label_col].values

    # Fit KNN classifier to approximate decision boundaries
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X, y)

    # Create a grid to plot decision boundaries
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                         np.linspace(y_min, y_max, 500))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = clf.predict(grid).reshape(xx.shape)

    # Plot decision regions
    unique_labels = np.unique(y)
    cmap = cm.get_cmap('jet', len(unique_labels))
    ax.contourf(xx, yy, Z, alpha=0.3, levels=len(unique_labels)-1, colors=[cmap(i) for i in range(len(unique_labels))])

    # Plot points
    for i, cluster in enumerate(unique_labels):
        pts = X[y == cluster]
        ax.scatter(pts[:, 0], pts[:, 1], color=cmap(i), edgecolor='k', s=40, label=f'Cluster {cluster}')

    ax.set_xlabel('PCA1')
    ax.set_ylabel('PCA2')
    ax.set_title(title)
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
### lets check thel PCA loadings
pca_df = pd.DataFrame(pca.components_, columns=features, index=[f'PC{i+1}' for i in range(pca.n_components_)]).T
pca_df

PC1 - Hoehe and soil moisture (+) and temp min, temp max, radi (-) ---> Climate and altitude gradient

PC2 - precipitation, temp mim, temp max (+) and Apfeldbluete and rad (-) ---> precipitation balance and climate

PC3 - rad (+)

In [ ]:
# median per cluster with features on x-axis and clusters side-by-side
medians = df_clustered[features + ['cluster']].groupby('cluster').median()
medians = medians[features]  # ensure feature order

medians_T = medians.T  # features as rows -> x-axis

ax = medians_T.plot(kind='bar', figsize=(12,6))
ax.set_xlabel('Feature')
ax.set_ylabel('Median value')
ax.set_title('Median feature values per cluster')
ax.legend(title='Cluster', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# median per cluster with features on x-axis and clusters side-by-side
medians = df_clustered_h[features + ['cluster']].groupby('cluster').median()
medians = medians[features]  # ensure feature order

medians_T = medians.T  # features as rows -> x-axis

ax = medians_T.plot(kind='bar', figsize=(12,6))
ax.set_xlabel('Feature')
ax.set_ylabel('Median value')
ax.set_title('Median feature values per cluster')
ax.legend(title='Cluster', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
plt.tight_layout()
plt.show()


Lets finally see the cluster on the map

In [ ]:
def plot_cluster_map_germany(df_clustered, germany, ax=None):
    gdf_points = gpd.GeoDataFrame(
        df_clustered,
        geometry=gpd.points_from_xy(df_clustered["Lon"], df_clustered["Lat"]),
        crs="EPSG:4326"
    )

    # Plot base map
    germany.plot(ax=ax, color="whitesmoke", alpha=0.6, zorder=1)
    germany.boundary.plot(ax=ax, linewidth=1, color='black', zorder=2)

    # Plot points
    gdf_points.plot(
        ax=ax,
        column="cluster",
        categorical=True,
        legend=True,
        cmap='jet',  # Using a colormap that works well with categorical data
        markersize=15,
        zorder=3
    )

    ax.set_axis_off()
    return ax

# Create figure and axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot KMeans clusters
plot_cluster_map_germany(df_clustered, germany, ax=ax1)
ax1.set_title("KMeans Clusters", pad=10, fontsize=12)

# Plot HDBSCAN clusters
plot_cluster_map_germany(final_hdbscan, germany, ax=ax2)
ax2.set_title("HDBSCAN Clusters", pad=10, fontsize=12)


In [ ]:
# Plot each feature as a separate map over Germany with normalized colorbars
features_to_plot = ['rad_global_apr', 'soil_moist_apr', 'precip_apr', 'temp_max_apr', 'temp_min_apr', 'Apfelbluete','Hoehe']

# Create a larger figure with 4 rows and 3 columns
fig, axes = plt.subplots(4, 3, figsize=(18, 14))
axes = axes.flatten()

for i, feature in enumerate(features_to_plot):
    ax = axes[i]
    germany.plot(ax=ax, color='white', edgecolor='black')
    vmin = gdf[gdf.Year==2023][feature].min()
    vmax = gdf[gdf.Year==2023][feature].max()
    gdf[gdf.Year==2023].plot(
        ax=ax,
        column=feature,
        cmap='jet',
        markersize=20,
        legend=True,
        alpha=0.8,
        vmin=vmin,
        vmax=vmax
    )
    ax.set_title(feature)
    ax.set_axis_off()

# Plot KMeans clusters in the 8th subplot
ax = axes[7]
germany.plot(ax=ax, color='white', edgecolor='black')
gdf_kmeans = gpd.GeoDataFrame(
    df_clustered,
    geometry=gpd.points_from_xy(df_clustered['Lon'], df_clustered['Lat']),
    crs="EPSG:4326"
)
gdf_kmeans.plot(
    ax=ax,
    column='cluster',
    categorical=True,
    cmap='jet',
    markersize=20,
    legend=True,
    alpha=0.8
)
ax.set_title('KMeans Clusters')
ax.set_axis_off()

# Plot HDBSCAN clusters in the 9th subplot
ax = axes[8]
germany.plot(ax=ax, color='white', edgecolor='black')
gdf_hdbscan = gpd.GeoDataFrame(
    final_hdbscan,
    geometry=gpd.points_from_xy(final_hdbscan['Lon'], final_hdbscan['Lat']),
    crs="EPSG:4326"
)
gdf_hdbscan.plot(
    ax=ax,
    column='cluster',
    categorical=True,
    cmap='jet',
    markersize=20,
    legend=True,
    alpha=0.8
)
ax.set_title('HDBSCAN Clusters')
ax.set_axis_off()

# Add PCA components to the plot
# Create a GeoDataFrame with PCA components
gdf_pca = gpd.GeoDataFrame(
    df_clustered.copy(),
    geometry=gpd.points_from_xy(df_clustered['Lon'], df_clustered['Lat']),
    crs="EPSG:4326"
)
gdf_pca['PC1'] = X_pca[:, 0]
gdf_pca['PC2'] = X_pca[:, 1]
gdf_pca['PC3'] = X_pca[:, 2]

# Plot PC1
ax = axes[9]
germany.plot(ax=ax, color='white', edgecolor='black')
gdf_pca.plot(
    ax=ax,
    column='PC1',
    cmap='jet',
    markersize=20,
    legend=True,
    alpha=0.8,
    vmin=gdf_pca['PC1'].min(),
    vmax=gdf_pca['PC1'].max()
)
ax.set_title('PC1 (Principal Component 1)')
ax.set_axis_off()

# Plot PC2
ax = axes[10]
germany.plot(ax=ax, color='white', edgecolor='black')
gdf_pca.plot(
    ax=ax,
    column='PC2',
    cmap='jet',
    markersize=20,
    legend=True,
    alpha=0.8,
    vmin=gdf_pca['PC2'].min(),
    vmax=gdf_pca['PC2'].max()
)
ax.set_title('PC2 (Principal Component 2)')
ax.set_axis_off()

# Plot PC3
ax = axes[11]
germany.plot(ax=ax, color='white', edgecolor='black')
gdf_pca.plot(
    ax=ax,
    column='PC3',
    cmap='jet',
    markersize=20,
    legend=True,
    alpha=0.8,
    vmin=gdf_pca['PC3'].min(),
    vmax=gdf_pca['PC3'].max()
)
ax.set_title('PC3 (Principal Component 3)')
ax.set_axis_off()

plt.tight_layout()
plt.show()

Let`s go do some other clustering 

Mean shift

In [ ]:
from sklearn import cluster

quantiles = [ 0.05,0.08, 0.1, 0.2, 0.3,0.5,1]

for q in quantiles:
    # estimate bandwidth and fit MeanShift on PCA data
    bw = cluster.estimate_bandwidth(X_pca, quantile=q)
    ms = cluster.MeanShift(bandwidth=bw, bin_seeding=True)
    ms.fit(X_pca)
    labels_ms = ms.labels_
    n_clusters = len(np.unique(labels_ms))
    print(f"quantile={q}  bandwidth={bw:.4f}  n_clusters={n_clusters}")

    # PCA scatter (3D if PCA has 3 components)
    plot_pca_scatter(X_pca, pca, features, color_labels=labels_ms, cmap='jet')

    # Map view for Germany
    df_ms = cluster_df.copy()
    df_ms['cluster'] = labels_ms

    fig, ax = plt.subplots(1, 1, figsize=(9, 6))
    plot_cluster_map_germany(df_ms, germany, ax=ax)
    ax.set_title(f"MeanShift clusters (quantile={q}, bw={bw:.3f})", pad=10)
    plt.tight_layout()
    plt.show()

Gaussian mixture

In [ ]:
from sklearn import mixture
from sklearn import cluster as skcluster
from sklearn.metrics import silhouette_score

n_list = [2, 3, 4]

for n_clusters in n_list:
    print(f"\n--- n_clusters = {n_clusters} ---")
    # --- Gaussian Mixture ---
    gmm = mixture.GaussianMixture(
        n_components=n_clusters,
        covariance_type="full",
        random_state=42,
    )
    labels_gmm = gmm.fit_predict(X_pca)
    try:
        sil_gmm = silhouette_score(X_pca, labels_gmm) if len(np.unique(labels_gmm)) > 1 else np.nan
    except Exception:
        sil_gmm = np.nan
    print(f"GMM: n_clusters={len(np.unique(labels_gmm))}, silhouette={sil_gmm:.3f}")

    # --- Spectral Clustering ---
    spectral = skcluster.SpectralClustering(
        n_clusters=n_clusters,
        eigen_solver="arpack",
        affinity="nearest_neighbors",
        random_state=42,
    )
    try:
        labels_spectral = spectral.fit_predict(X_pca)
        sil_spec = silhouette_score(X_pca, labels_spectral) if len(np.unique(labels_spectral)) > 1 else np.nan
    except Exception as e:
        labels_spectral = np.full(len(X_pca), -1)
        sil_spec = np.nan
        print(f"Spectral failed for n={n_clusters}: {e}")
    print(f"Spectral: n_clusters={len(np.unique(labels_spectral))}, silhouette={sil_spec:.3f}")

    # --- 3D side-by-side PCA plots ---
    if X_pca.shape[1] < 3:
        print("PCA has less than 3 components, skipping 3D plots.")
    else:
        fig = plt.figure(figsize=(14, 6))

        # GMM subplot
        ax1 = fig.add_subplot(121, projection='3d')
        sc1 = ax1.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
                          c=labels_gmm, cmap='jet', s=35, alpha=0.8)
        arrow_scale = np.max(np.abs(X_pca)) * 1.2
        for i, feature in enumerate(features):
            ax1.quiver(0, 0, 0,
                       pca.components_[0, i] * arrow_scale,
                       pca.components_[1, i] * arrow_scale,
                       pca.components_[2, i] * arrow_scale,
                       color='r', alpha=0.6, linewidth=1)
            ax1.text(pca.components_[0, i] * arrow_scale,
                     pca.components_[1, i] * arrow_scale,
                     pca.components_[2, i] * arrow_scale,
                     feature, color='r', fontsize=9, ha='center', va='center')
        ax1.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
        ax1.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
        ax1.set_zlabel(f"PC3 ({pca.explained_variance_ratio_[2]*100:.1f}% var)")
        ax1.view_init(elev=30, azim=120)
        ax1.set_title(f"GMM clusters (n={n_clusters})\nSilhouette={sil_gmm:.3f}")
        fig.colorbar(sc1, ax=ax1, pad=0.05, shrink=0.6)

        # Spectral subplot
        ax2 = fig.add_subplot(122, projection='3d')
        sc2 = ax2.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
                          c=labels_spectral, cmap='jet', s=35, alpha=0.8)
        for i, feature in enumerate(features):
            ax2.quiver(0, 0, 0,
                       pca.components_[0, i] * arrow_scale,
                       pca.components_[1, i] * arrow_scale,
                       pca.components_[2, i] * arrow_scale,
                       color='r', alpha=0.6, linewidth=1)
            ax2.text(pca.components_[0, i] * arrow_scale,
                     pca.components_[1, i] * arrow_scale,
                     pca.components_[2, i] * arrow_scale,
                     feature, color='r', fontsize=9, ha='center', va='center')
        ax2.set_xlabel("PC1")
        ax2.set_ylabel("PC2")
        ax2.set_zlabel("PC3")
        ax2.view_init(elev=30, azim=120)
        ax2.set_title(f"Spectral clusters (n={n_clusters})\nSilhouette={sil_spec:.3f}")
        fig.colorbar(sc2, ax=ax2, pad=0.05, shrink=0.6)

        plt.tight_layout()
        plt.show()

    # --- Map side-by-side: GMM vs Spectral ---
    df_gmm = cluster_df.copy()
    df_gmm['cluster'] = labels_gmm

    df_spectral = cluster_df.copy()
    df_spectral['cluster'] = labels_spectral

    fig, (ax1_map, ax2_map) = plt.subplots(1, 2, figsize=(15, 6))
    plot_cluster_map_germany(df_gmm, germany, ax=ax1_map)
    ax1_map.set_title(f'GMM clusters (n={n_clusters})', pad=10, fontsize=12)

    plot_cluster_map_germany(df_spectral, germany, ax=ax2_map)
    ax2_map.set_title(f'Spectral clusters (n={n_clusters})', pad=10, fontsize=12)

    plt.tight_layout()
    plt.show()
